#Load Data

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F

In [4]:
from matplotlib import pyplot as plt
import numpy as np

In [5]:
data_set_dir = "/content/drive/My Drive/Machine Learning/Tubes/Dataset"
transform = transforms.Compose([transforms.Resize((64, 64)), transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

#Step 2


In [6]:
dataset = torchvision.datasets.ImageFolder(data_set_dir, transform=transform)
batch_size = 60

In [7]:
print(len(dataset))
from torch.utils.data import random_split

4094


In [8]:
val_size = 800
trains_size = len(dataset) - val_size
train_dataset, val_dataset = random_split(dataset, [trains_size, val_size])

In [9]:
print(len(train_dataset))
print(len(val_dataset))

3294
800


In [10]:
from torch.utils.data.dataloader import DataLoader

In [11]:
train_dl = DataLoader(train_dataset, batch_size, shuffle=True)
val_dl = DataLoader(val_dataset, batch_size*2)

In [12]:
class ImageClassification(nn.Module):

    def train_step(self, batch):
        images, labels = batch
        out = self(images)
        loss = F.cross_entropy(out, labels)
        return loss

    def validation_step(self, batch):
        images, labels = batch
        out = self(images)
        loss = F.cross_entropy(out, labels)
        acc = accuracy(out, labels)

        return {'val_loss': loss.detach(), 'val_acc': acc}


    def validation_epoch_end(self, outputs):
        batch_losses = [x['val_loss'] for x in outputs]
        epoch_loss = torch.stack(batch_losses).mean()  # Combine losses
        batch_accs = [x['val_acc'] for x in outputs]
        epoch_acc = torch.stack(batch_accs).mean()  # Combine accuracies
        return {'val_loss': epoch_loss.item(), 'val_acc': epoch_acc.item()}


    def epoch_end(self, epoch, result):
        print("Epoch [{}], train_loss: {:.4f}, val_loss: {:.4f}, val_acc: {:.4f}".format(
            epoch, result['train_loss'], result['val_loss'], result['val_acc']))

In [13]:
class MaskClassification(ImageClassification):
    def __init__(self):
        super().__init__()
        self.network = nn.Sequential(

            # LAYER 1
            nn.Conv2d(3, 32, kernel_size=3, padding=(2, 2), stride=2),
            nn.ReLU(),
            nn.Conv2d(32, 64, kernel_size=3, padding=(2, 2), stride=2),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),
            nn.Dropout(0.25),

            # LAYER 2
            nn.Conv2d(64, 64, kernel_size=3, padding=(2, 2), stride=2),
            nn.ReLU(),
            nn.Conv2d(64, 64, kernel_size=3, padding=(2, 2), stride=2),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),
            nn.Dropout(0.25),

            # LAYER 3
            nn.Conv2d(64, 128, kernel_size=3, padding=(2, 2), stride=2),
            nn.ReLU(),
            nn.Conv2d(128, 128, kernel_size=3, padding=(2, 2), stride=2),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),
            nn.Dropout(0.5),

            # LAST LAYER
            nn.Flatten(),
            nn.Linear(128, 1024),
            nn.ReLU(),
            nn.Linear(1024, 512),
            nn.ReLU(),
            nn.Linear(512, 3)
        )

    def forward(self, xb):
        return self.network(xb)

In [14]:
model = MaskClassification()

def accuracy(outputs, labels):
    _, preds = torch.max(outputs, dim=1)
    return torch.tensor(torch.sum(preds == labels).item() / len(preds))


@torch.no_grad()
def evaluate(model, val_loader):
    model.eval()
    outputs = [model.validation_step(batch) for batch in val_loader]
    return model.validation_epoch_end(outputs)


def fit(epochs, lr, model, train_loader, val_loader, opt_func=torch.optim.SGD):

    history = []
    optimizer = opt_func(model.parameters(), lr)
    for epoch in range(epochs):

        model.train()
        train_losses = []
        for batch in train_loader:
            loss = model.train_step(batch)
            train_losses.append(loss)
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()


        result = evaluate(model, val_loader)
        result['train_loss'] = torch.stack(train_losses).mean().item()
        model.epoch_end(epoch, result)
        history.append(result)

    return history

In [17]:
num_of_epoch = 20
optimizer_func = torch.optim.Adam
lr = 0.001

In [18]:
history = fit(num_of_epoch, lr, model, train_dl, val_dl, optimizer_func)

/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)
/usr/local/lib/python3.7/dist-packages/PIL/Image.py:960: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  "Palette images with Transparency expressed in bytes should be "


Epoch [0], train_loss: 0.7490, val_loss: 0.6919, val_acc: 0.5214
Epoch [1], train_loss: 0.6481, val_loss: 0.4989, val_acc: 0.7589
Epoch [2], train_loss: 0.4225, val_loss: 0.3238, val_acc: 0.8786
Epoch [3], train_loss: 0.3199, val_loss: 0.2905, val_acc: 0.8952
Epoch [4], train_loss: 0.2863, val_loss: 0.2349, val_acc: 0.9107
Epoch [5], train_loss: 0.2158, val_loss: 0.1654, val_acc: 0.9387
Epoch [6], train_loss: 0.1685, val_loss: 0.1509, val_acc: 0.9423
Epoch [7], train_loss: 0.1545, val_loss: 0.1311, val_acc: 0.9536
Epoch [8], train_loss: 0.1247, val_loss: 0.1828, val_acc: 0.9411
Epoch [9], train_loss: 0.1094, val_loss: 0.1459, val_acc: 0.9560
Epoch [10], train_loss: 0.0922, val_loss: 0.1565, val_acc: 0.9452
Epoch [11], train_loss: 0.0848, val_loss: 0.1921, val_acc: 0.9500
Epoch [12], train_loss: 0.0738, val_loss: 0.2013, val_acc: 0.9458
Epoch [13], train_loss: 0.0655, val_loss: 0.1676, val_acc: 0.9554
Epoch [14], train_loss: 0.0565, val_loss: 0.1294, val_acc: 0.9625
Epoch [15], train_lo

In [19]:
import pathlib
root=pathlib.Path("/content/drive/My Drive/Machine Learning/Tubes/Dataset")
classes=sorted([j.name.split('/')[-1] for j in root.iterdir()])

In [20]:
print(classes)

['with_mask', 'without_mask']


In [21]:
#Transforms

transformer=transforms.Compose([
    transforms.Resize((64,64)),
    transforms.ToTensor(),  #0-255 to 0-1, numpy to tensors
    transforms.Normalize([0.5,0.5,0.5], # 0-1 to [-1,1] , formula (x-mean)/std
                        [0.5,0.5,0.5])
])

In [22]:
import glob
from PIL import Image
from torch.autograd import Variable

# Prediction Function

def prediction(img_path,transformer):
    
    image=Image.open(img_path)
    
    image_tensor=transformer(image).float()
    
    
    image_tensor=image_tensor.unsqueeze_(0)
    
    if torch.cuda.is_available():
        image_tensor.cuda()
        
    input=Variable(image_tensor)
    
    
    output=model(input)
    
    index=output.data.numpy().argmax()
    
    pred=classes[index]
    
    return pred

In [27]:
# How to predict

# With Mask
image_path1 = "/content/drive/My Drive/maksssksksss6.png"
result1 = prediction(image_path1,transformer)

# Mask weared incorrect
# image_path2 = "../input/face-mask-detection/Dataset/mask_weared_incorrect/1002.png"
# result2 = prediction(image_path2,transformer)

# Without Mask
# image_path3 = "../input/face-mask-detection/Dataset/without_mask/1006.png"
# result3 = prediction(image_path3,transformer)

result1

'with_mask'

In [31]:
# Save the model

import torch
FILE = "/content/drive/My Drive/Machine Learning/Tubes/Model 1/model_1.h5"
torch.save(model, FILE)

In [ ]:
import json

target_dir = '/content/drive/My Drive/Machine Learning/Tubes/Model 1'

# simpan history model 1
history_dict = history.history
json.dump(history_dict, open(target_dir + '/history_model_1.json', 'w'))

# simpan model 11 dan weight-nya
model.save(target_dir + '/model_1.h5')
model.save_weights(target_dir + '/model_1_weights.h5')